# Trabajo Práctico Nro 1 - Redes Neuronales, ITBA 2023
Autor: Pablo Smolkin

Legajo 59523
***

## Problema de Clasificación 

Se utilizará el dataset de Fashion MNIST, que se encuentra en la [competencia de Kaggle](https://www.kaggle.com/t/5f2641ef201d4e8aa500d4876fb2b64e) que quedará abierta hasta el final de la cursada y otorgará puntos extras a los grupos ganadores. La competencia puede hacerse por grupos (la cantidad de integrantes que se deseen pero se reparten los puntos al final). La entrega del TP es individual.

### Imports

In [ ]:
from softmax_helper import *

##### Old cell testing

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, RocCurveDisplay, roc_auc_score, f1_score, confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from datetime import datetime

### Load Dataset

In [ ]:
x_data, y_data, class_names = load_fmnist_data()

##### Old cell testing

In [ ]:
x_data = np.load('../AssignmentGoodies/train_images.npy')
print(f"x data Shape: {x_data.shape}")

y_data = pd.read_csv('../AssignmentGoodies/train_labels.csv').to_numpy()[:,0]
print(f"y data Shape: {y_data.shape}")

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### 1. **EDA:** 
Visualizar algunas instancias de cada clase. Hacer histogramas de la distribución de intensidades para cada clase, cualquier otra propuesta es bienvenida.

In [ ]:
fmnist_eda_plots(x_data, y_data, class_names)

##### Old cell testing

In [ ]:
# Find indexes of each class instance in y_data
class_examples = [np.where(y_data == i) for i in range(10)]

In [ ]:
# Plot 3 random examples of each class
plt.figure(figsize=(20, 6))
for i in range(10):
    for j in range(3):
        plt.subplot(3, 10, i+1 + j*10)
        plt.imshow(x_data[class_examples[i][0][np.random.randint(0, 6000)], ...])
        plt.axis('off')
    plt.title(f'{class_names[i]}', pad=245)
plt.show()

In [ ]:
# Plot brightness mean of each class
plt.figure(figsize=(20, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(np.mean(x_data[class_examples[i][0], ...], axis=0))
    plt.axis('off')
    plt.title(f'{class_names[i]}')

In [ ]:
# Plot brightness distribution of each class
plt.figure(figsize=(20, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.hist(np.mean(x_data[class_examples[i][0], ...], axis=(1,2)), color=plt.cm.Accent(i%8), bins=50)
    plt.grid()
    plt.xlim(0, 255)
    plt.ylim(0, 650)
    plt.minorticks_on()
    plt.tight_layout()
    plt.title(f'{class_names[i]}')

### 2. **Modelos:** 
Entrenar un modelo de clasificación en las 10 clases de Fashion MNIST. Regresión softmax y MLP. En el caso que corresponda probar y comparar: distintas funciones de activación, distintos optimizadores, distintas funciones de costo, distintos tamaños de red (cantidad de capas y tamaño de cada capa), learning rate, batch_size, dropout rates, batch_normalization (si/no), inicializaciones de pesos (glorot_uniform, glorot_normal, normal(0,1), normal(0,0.001)).

Métricas a reportar para cada entrenamiento de interés: Accuracy como métrica principal.
Como métricas secundarias: curva ROC, área bajo la curva ROC, F1-score, Precisión y
Recall. Enunciar para el F1-score y el área bajo la curva ROC cómo se calculan los
promedios macro y micro. Justificar cuál de los dos es el que corresponde reportar en este
problema.

Para un buen modelo obtenido en el punto anterior (así si lo mejoran no tienen que repetir
este punto), variar los hiperparámetros de a uno y graficar:

* Accuracy vs [LEARNING_RATE, BATCH_SIZE, OPTIMIZADORES, ACTIVACIONES, DROPOUT_RATE, BATCH_NORMALIZATION, INICIALIZACIONES DE PESOS]

* Iteraciones de entrenamiento vs [LEARNING_RATE, BATCH_SIZE, OPTIMIZADORES, ACTIVACIONES, DROPOUT_RATE, BATCH_NORMALIZATION, INICIALIZACIONES DE PESOS]

Intente justificar los gráficos obtenidos. Valores mínimos esperados para cada modelo: SoftmaxReg: .70, MLP: .83

#### Softmax

##### Function Testing

In [ ]:
# Loading TensorBoard for learning logging
%load_ext tensorboard

In [ ]:
softReg, x_train, x_valid, y_train, y_valid = create_fmnist_model(
    x_data,
    y_data,
    metrics=['accuracy','AUC','Precision','Recall'],
    loss='categorical_crossentropy',
    optimizer='Adam',
    learning_rate=0.0001,
    train_valid_proportion=1/3,
    random_state_seed=10,
    verbose=False
)

In [ ]:
history, metrics_df = run_model(
    model=softReg,
    x_train=x_train,
    x_valid=x_valid,
    y_train=y_train,
    y_valid=y_valid,
    batch_size=32,
    max_epochs=100,
    es_config = {
        'monitor': 'val_accuracy',
        'patience': 10,
        'mode': 'max',
        'restore_best_weights': True
    },
    show_metrics=True,
    verbose=False
)

##### Old Cell Testing

In [ ]:
# Normalize the data
scaler = StandardScaler()
x_stand = scaler.fit_transform(x_data.reshape(-1, 784)).reshape(-1, 28, 28)

In [ ]:
# Split the train_valid sub-dataset into train and valid
x_train, x_valid, y_train, y_valid = train_test_split(x_stand, y_data, test_size=1/3, random_state=10, shuffle=True)

In [ ]:
print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)

In [ ]:
# Make sparce label vectors
y_sparse_train = np.zeros([40000,10])
y_sparse_valid = np.zeros([20000,10])
for idx in range(40000):
    y_sparse_train[idx,y_train[idx]] = 1
for idx in range(20000):
    y_sparse_valid[idx,y_valid[idx]] = 1

In [ ]:
# Loading TensorBoard for learning logging
%load_ext tensorboard

In [ ]:
# Softmax model
softReg = Sequential()
softReg.add(Flatten(input_shape=(28,28)))
softReg.add(Dense(10, activation="softmax"))
softReg.summary()
metrics = ['accuracy','AUC','Precision','Recall']
softReg.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.0001),metrics=metrics)

In [ ]:
# Configuring TensorBoard to log learning process
log_dir = "logs/softmax/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True, mode='max', restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = softReg.fit(x=x_train,
                      y=y_sparse_train,
                      validation_data=(x_valid, y_sparse_valid),
                      batch_size=32,
                      epochs=100,
                      callbacks=[earlyStopping, tensorboard_callback],
                      use_multiprocessing=True)

In [ ]:
best_epoch_train_metrics = {metric: history.history[metric][earlyStopping.best_epoch] for metric in history.history.keys() if not metric.startswith('val_')}
best_epoch_val_metrics = {metric.removeprefix('val_'): history.history[metric][earlyStopping.best_epoch] for metric in history.history.keys() if metric.startswith('val_')}

f1_train = f1_score(y_train, clasify_maxprob(softReg.predict(x_train, verbose=False, use_multiprocessing=True)), average='macro')
f1_val = f1_score(y_valid, clasify_maxprob(softReg.predict(x_valid, verbose=False, use_multiprocessing=True)), average='macro')

best_epoch_train_metrics['f1'] = f1_train
best_epoch_val_metrics['f1'] = f1_val

metrics_df = pd.DataFrame({'train': best_epoch_train_metrics, 'val': best_epoch_val_metrics}).sort_index(key=lambda x: x.str.lower())

metrics_df

In [ ]:
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.legend()
plt.show()
maxAccuracy = max(history.history["accuracy"])
maxValAccuracy = max(history.history["val_accuracy"])
print(f"Max Accuracy: {maxAccuracy:.4f}")
print(f"Max Val Accuracy: {maxValAccuracy:.4f}")

#### MLP

In [ ]:
# Normalize the data
scaler = StandardScaler()
x_stand = scaler.fit_transform(x_data.reshape(-1, 784)).reshape(-1, 28, 28)

In [ ]:
# Split the train_valid sub-dataset into train and valid
x_train, x_valid, y_train, y_valid = train_test_split(x_stand, y_data, test_size=1/3, random_state=10, shuffle=True)

In [ ]:
print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)

In [ ]:
# Make sparce label vectors
y_sparse_train = np.zeros([40000,10])
y_sparse_valid = np.zeros([20000,10])
for idx in range(40000):
    y_sparse_train[idx,y_train[idx]] = 1
for idx in range(20000):
    y_sparse_valid[idx,y_valid[idx]] = 1

In [ ]:
# Softmax model
model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))
model.summary()
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.0001),metrics=["accuracy"])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='max')
history = model.fit(x= x_train, y = y_sparse_train, validation_data=(x_valid, y_sparse_valid), batch_size = 32, epochs=100, callbacks=[earlyStopping])

In [ ]:
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.legend()
plt.show()
maxAccuracy = max(history.history["accuracy"])
maxValAccuracy = max(history.history["val_accuracy"])
print(f"Max Accuracy: {maxAccuracy:.4f}")
print(f"Max Val Accuracy: {maxValAccuracy:.4f}")

### 3. 
El mejor modelo debe ser reentrenado agregando una capa de tamaño 2 antes de la de salida. Mapear los datos de entrada a la salida de la capa agregada y graficarla con los labels correspondientes. Superponer en el mismo gráfico los umbrales de decisión. Para obtener los umbrales de decisión se debe armar una grilla de puntos que debe darse como input a la última capa de la red neuronal (softmax) y tomar el argumento máximo como clase correspondiente.
   
Se alienta a intentar aumentar los datos de entrenamiento haciendo espejamiento de las imágenes en las situaciones que tenga sentido hacerlo. 

Una vez entregado el TP se pueden seguir subiendo predicciones de modelos nuevos a Kaggle hasta el final de la cursada.

### 4. Para un buen conjunto de hiperparámetros de un MLP obtenido en 2, entrenar de la siguiente manera:
* Salvar la red inicializada con pesos aleatorios (antes de entrenar). La llamaremos red 1.
* Entrenar la red. La llamaremos red 2.
* Comparar los pesos obtenidos en red 2 con los pesos de inicialización (red 1) y marcar a aquel 50% que más haya variado.
* Sobre la red 1 eliminar los pesos que no hayan sido marcados en el punto anterior. La llamaremos red 3.
* Sin entrenar, medir el accuracy de red 3.
* Entrenar la red 3 (de 1/2 del tamaño original) y comparar la métrica con la métrica obtenida con red 2.
* Discutiremos los resultados en clase.

***

## Problema de Regresión

***

## Testing Code

In [ ]:
x_data = np.load('../AssignmentGoodies/train_images.npy')
print(x_data.shape)

y_data = pd.read_csv('../AssignmentGoodies/train_labels.csv').to_numpy()[:,0]
print(y_data.shape)

In [ ]:
y_data.shape

In [ ]:
y_data[0]

In [ ]:
plt.imshow(x_data[0,...])

In [ ]:
from sklearn import model_selection

In [ ]:
# Split the train_valid sub-dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_data, y_data, test_size=1/3, random_state=10, shuffle=True)

In [ ]:
x_train, x_valid = x_train / 255.0, x_valid / 255.0

In [ ]:
print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)

In [ ]:
y_sparse_train = np.zeros([40000,10])
y_sparse_valid = np.zeros([20000,10])

In [ ]:
for idx in range(40000):
    y_sparse_train[idx,y_train[idx]] = 1

for idx in range(20000):
    y_sparse_valid[idx,y_valid[idx]] = 1

In [ ]:
y_sparse_train[0]

In [ ]:
y_sparse_valid[0]

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model =  Sequential()

In [ ]:
model.add(Flatten(input_shape=(28,28)))

In [ ]:
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.0001),metrics=["accuracy"])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
history = model.fit(x= x_train, y = y_sparse_train, validation_data=(x_valid, y_sparse_valid), batch_size = 32, epochs=100, callbacks=[earlyStopping])

In [ ]:
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.legend()
plt.show()
maxAccuracy = max(history.history["accuracy"])
maxValAccuracy = max(history.history["val_accuracy"])
print(f"Max Accuracy: {maxAccuracy:.4f}")
print(f"Max Val Accuracy: {maxValAccuracy:.4f}")

### Con Conv2D

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import model_selection

fashion_mnist = np.load('../AssignmentGoodies/train_images.npy')
print(fashion_mnist.shape)
x_data = fashion_mnist
y_data = pd.read_csv('../AssignmentGoodies/train_labels.csv').to_numpy()[:,0]
print(y_data.shape)

# Split the train_valid sub-dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_data, y_data, test_size=1/3, random_state=10, shuffle=True)

x_train, x_valid = x_train / 255.0, x_valid / 255.0

y_sparse_train = np.zeros([40000,10])
y_sparse_valid = np.zeros([20000,10])

for idx in range(40000):
    y_sparse_train[idx,y_train[idx]] = 1

for idx in range(20000):
    y_sparse_valid[idx,y_valid[idx]] = 1

model = Sequential()
model.add(Conv2D(32, 3, activation='relu', input_shape=(28,28,1)))
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(128, activation="relu", kernel_initializer="glorot_normal"))
model.add(Dense(10, activation="softmax", kernel_initializer="glorot_normal"))

model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.0001, amsgrad=True),metrics=["accuracy"])

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

In [ ]:
history = model.fit(x= x_train, y = y_sparse_train, validation_data=(x_valid, y_sparse_valid), batch_size = 32, epochs=100, callbacks=[earlyStopping])

In [ ]:
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.legend()
plt.show()
maxAccuracy = max(history.history["accuracy"])
maxValAccuracy = max(history.history["val_accuracy"])
print(f"Max Accuracy: {maxAccuracy:.4f}")
print(f"Max Val Accuracy: {maxValAccuracy:.4f}")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

x_train_c = x_train[..., tf.newaxis]
x_valid_c = x_valid[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train_c, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_valid_c, y_valid)).batch(32)

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='test_accuracy')


@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


EPOCHS = 30

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))